# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
! pip install gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
file = '../Output_Data/CityWeatherData.csv'
vacation_df = pd.read_csv(file)

# Preview of the DataFrame
vacation_df.head()

,City Name,Country Code,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness
0,Butaritari,KI,3.0707,172.7902,79.48,82,9.64,60
1,Kapaa,US,22.0752,-159.3190,67.08,88,23.02,90
2,Klaksvík,FO,62.2266,-6.5890,42.80,100,24.16,90
3,Castro,BR,-24.7911,-50.0119,75.88,65,5.26,0
4,Turukhansk,RU,65.8167,87.9833,-14.76,89,9.57,100


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

#Create Locations
locations = vacation_df[["Latitude", "Longitude"]]
humidity = vacation_df["Humidity"].astype(float)

#Create Heat Layers
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity = 100, point_radius = 5)

fig.add_layer(heat_layer)

#Disply figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create criteria for the perfect vacation climate
ideal_temperature = (vacation_df['Temperature'] < 85) & (vacation_df['Temperature'] > 65)
ideal_windSpeed = vacation_df['Wind Speed'] < 15
ideal_cloudiness = vacation_df['Cloudiness'] < 10
final_criteria = ideal_temperature & ideal_windSpeed & ideal_cloudiness 

# Use boolean indexing to filter the weather_df dataframe
ideal_weather_df = vacation_df[final_criteria]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()
ideal_weather_df.head()

,index,City Name,Country Code,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness
0,3,Castro,BR,-24.7911,-50.0119,75.88,65,5.26,0
1,18,Tura,IN,25.5198,90.2201,71.37,42,3.65,0
2,20,Hong Kong,HK,22.2855,114.1577,65.93,86,10.56,2
3,26,Shakawe,BW,-18.3667,21.8500,75.76,56,3.87,1
4,73,Corbélia,BR,-24.7989,-53.3067,79.61,54,8.55,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather_df
# Add column for Hotel Name - Note that we used "" to specify initial entry.
hotel_df['Hotel Name'] = ""
hotel_df.head()

,index,City Name,Country Code,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness,Hotel Name
0,3,Castro,BR,-24.7911,-50.0119,75.88,65,5.26,0,
1,18,Tura,IN,25.5198,90.2201,71.37,42,3.65,0,
2,20,Hong Kong,HK,22.2855,114.1577,65.93,86,10.56,2,
3,26,Shakawe,BW,-18.3667,21.8500,75.76,56,3.87,1,
4,73,Corbélia,BR,-24.7989,-53.3067,79.61,54,8.55,0,


In [7]:
# params dictionary to update each iteration
hotellist = []

for i in range(len(hotel_df)):
        
    params = {
    "radius": 5000,
    "keyword": "Hotel",
    "key": g_key
    }
    
    lat = hotel_df.loc[i]['Latitude']
    lng = hotel_df.loc[i]['Longitude']
    
    params['location'] = f'{lat}, {lng}'
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    hotel_json = requested.json()
                          
    
    try:
        hotellist.append(hotel_json['results'][0]['name'])
    except:
        hotellist.append("")
hotel_df["Hotel Name"] = hotellist
hotel_df = hotel_df.dropna(how='any')
hotel_df


,index,City Name,Country Code,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness,Hotel Name
0,3,Castro,BR,-24.7911,-50.0119,75.88,65,5.26,0,Buganville Palace Hotel
1,18,Tura,IN,25.5198,90.2201,71.37,42,3.65,0,"Dongrama Home Stay, Tura Meghalaya"
2,20,Hong Kong,HK,22.2855,114.1577,65.93,86,10.56,2,The Harbourview
3,26,Shakawe,BW,-18.3667,21.8500,75.76,56,3.87,1,Athoma Guest House
4,73,Corbélia,BR,-24.7989,-53.3067,79.61,54,8.55,0,Hotel Peroza
5,86,Noumea,NC,-22.2763,166.4572,68.00,94,2.30,0,Le Méridien Noumea Resort & Spa
6,114,Dalbandin,PK,28.8947,64.4101,69.96,16,6.46,0,Al-Dawood Hotel & Restaurant
7,115,Bacungan,PH,9.9106,118.7022,76.73,82,3.67,0,B&R HOSTEL
8,156,Isangel,VU,-19.5500,169.2667,75.76,82,12.35,7,Sunset Bungalows
9,157,Saint-Philippe,RE,-21.3585,55.7679,74.77,74,5.75,0,Les Embruns Du Baril


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country Code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))